# Passing the Pydantic model directly in the API call to LLMs

Import all required libraries

In [25]:
# Import packages
from pydantic import BaseModel, Field, EmailStr
from typing import List, Literal, Optional
from openai import OpenAI
# instructor is a library for extracting structured data from Large Language Models (LLMs). It is built on top of Pydantic.
# instructor extracts the JSON schema from a Pydantic model and passes it in the prompt. It handles retries and parsing of the response.
# Instructor takes a 'response_model' parameter in the API call, which is a Pydantic model that defines the structure of the expected response.
import instructor
import anthropic
from dotenv import load_dotenv
from datetime import date
import os

load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
# Define the UserInput model for customer support queries
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        # Default value is None
        None,
        description="5-digit order number (cannot start with 0)",
        #Greater or equal to 10000 and less than or equal to 99999
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

# Define the CustomerQuery model that inherits from UserInput. It adds fields that will be populated by the LLM for priority, category, complaint status, and tags.
class CustomerQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high"
    )
    category: Literal[
        'refund_request', 'information_request', 'other'
    ] = Field(..., description="Query category")
    is_complaint: bool = Field(
        ..., description="Whether this is a complaint"
    )
    tags: List[str] = Field(..., description="Relevant keyword tags")

In [27]:
# Create a sample user input as a JSON string
user_input_json = '''{
    "name": "Ula Dobra",
    "email": "ula.dobra@podlasem.com",
    "query": "I would like to know the status of my order.",
    "order_id": 87647,
    "purchase_date": "2025-01-01"
}'''



In [28]:
#Validate the user input against the UserInput model using the model_validate_json method
user_input = UserInput.model_validate_json(user_input_json)
print("User Input:")
print(user_input)


User Input:
name='Ula Dobra' email='ula.dobra@podlasem.com' query='I would like to know the status of my order.' order_id=87647 purchase_date=datetime.date(2025, 1, 1)


When using this method, we'll rely on Instructor to pass the JSON schema of the Pydantic model in the API call. 

In [29]:
#No mention of the desired data structure in the prompt. Instructor will handle that.
prompt = (
    f"Analyze the following customer query {user_input} "
    f"and provide a structured response."
)

In [30]:

# Use Anthropic with Instructor to get structured output
anthropic_client = instructor.from_anthropic(
    anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
)

response = anthropic_client.messages.create(
    model="claude-3-7-sonnet-latest",  
    max_tokens=1024,
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ],
    response_model=CustomerQuery  
)

In [31]:
print(response)

name='Ula Dobra' email='ula.dobra@podlasem.com' query='I would like to know the status of my order.' order_id=87647 purchase_date=datetime.date(2025, 1, 1) priority='medium' category='information_request' is_complaint=False tags=['order status', 'inquiry']
